# Text Classification with Transformers

### Objective:
1. Familiarize yourself with huggingface datasets and models
2. Learn to perform Binary and Multi Class Classification

### Problem Statement:
Authorship Profiling is the task of learning / predicting certain characteristics of the author with respect to demographics etc. It is hypothesised that in certain cases, the profile of an author with respect to say Gender can differenciate their style of writing.

**Task1 - Gender Prediction (Binary Classification)**: Does gender influence their written text?

**Task2 - Age Group Prediction (Multi Class Classification)**: Does the age group of a person influence their written text?

**Dataset**:  [Blog Authorship Corpus](https://huggingface.co/datasets/blog_authorship_corpus)

Change Runtime to GPU if possible

In [1]:
import torch
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import Pipeline
import evaluate
import numpy as np
from numpy import zeros
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import os
import pandas as pd
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
from transformers import pipeline

## Loading Data


In [2]:
raw_dataset = (load_dataset('blog_authorship_corpus', split='train', trust_remote_code=True)
        .train_test_split(train_size=1000, test_size=100, seed=42))

In [3]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'date', 'gender', 'age', 'horoscope', 'job'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['text', 'date', 'gender', 'age', 'horoscope', 'job'],
        num_rows: 100
    })
})

In [4]:
raw_dataset["train"][10]

{'text': "ahah oh yeah. hope got pple join canoeing. muahhaha. sian loh if no one join. or if all the retards join. i'm gonna fsking kill myself. lololol.",
 'date': '02,January,2004',
 'gender': 'male',
 'age': 17,
 'horoscope': 'Sagittarius',
 'job': 'indUnk'}

## Task1.1 - Gender Prediction (Binary Classification )
Can you predict the gender of a person from a piece of written text?


In [5]:
labels = ["female", "male"]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

In [6]:
label2id

{'female': 0, 'male': 1}

In [7]:
id2label

{0: 'female', 1: 'male'}

In [8]:
def tokenize_function(batch, tokenizer,label2id ):#= tokenizer
    tokenized_batch = tokenizer(batch["text"],padding=True, max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
    tokenized_batch["labels"] = [label2id.get(label) for label in batch["gender"]] #label
    return tokenized_batch

Models can come in cased or uncased version. Uncased models convert to lower case and remove accents before progressing while all this information is retained for cased models. Cased models may be more suited to tasks such as NER and POS tagging wehre such information is important

In [9]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [10]:
tokenized_dataset = raw_dataset.map(tokenize_function, batched=True, fn_kwargs={"tokenizer": tokenizer, "label2id":label2id},num_proc=4, remove_columns=raw_dataset['train'].column_names) #use fn_kwargs to pass any arguments to the tokenizing function

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]

In [11]:
tokenized_dataset.set_format(type="torch")

In [12]:
tokenized_dataset["train"][0]

{'input_ids': tensor([  101,  2035,  2002,  2515,  2012,  2147,  1012,  1012,  1012,  1012,
          1008,  5959,  2010,  2502,  4012, 12031,  6411,  1012,  1008,  3422,
          5691,  1012,  1012,  1012,  1012,  1008,  1998,  8692,  1012,  1012,
          1012,  1012,  1008,  1998,  2694,  3065,  1012,  1012,  1012,  1012,
          1008,  2673,  2694,   999,   999,   999,  1006,  8840,  2140,  1007,
          1008, 10053,  1008,  2980,  2030,  2025,  1012,  1012,  1012,  1006,
          1051,  9541,  3336,  2115, 15488, 23212,  2078,  1005,   999,  1007,
          1024,  1066,  1008,  1024,  1024, 14163,  4213,  2232,  1024,  1024,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

[Data Collators](https://huggingface.co/docs/transformers/main_classes/data_collator) are used to batch together input data to take care of padding (E.g. *DataCollatorWithPadding*), batching, dynamic masking (E.g. *DataCollatorForLanguageModeling*) or handling special token requirements (E.g. *DataCollatorForTokenClassification*).

In [13]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding = True) #pads to the max sequence length in a batch

In [14]:
accuracy = evaluate.load("accuracy") #f1, precision, 
#https://huggingface.co/docs/evaluate/choosing_a_metric

In [15]:
def compute_metrics(eval_pred):
    predictions_distribution, labels = eval_pred
    predictions = np.argmax(predictions_distribution, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [16]:
#https://github.com/huggingface/transformers/blob/main/src/transformers/models/auto/modeling_auto.py
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, id2label=id2label, label2id=label2id) #problem_type="multi_label_classification"

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
#https://github.com/huggingface/transformers/blob/main/src/transformers/training_args.py
training_args = TrainingArguments(
    report_to=None,
    output_dir="models/"+model_name + "_blog_authorship_gender",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/scratch/local/35422920/ipykernel_150992/552906714.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [19]:
df_train = raw_dataset["train"].to_pandas()
df_test = raw_dataset["test"].to_pandas()

In [20]:
df_train.head()

,text,date,gender,age,horoscope,job
0,All he does at work.... *Enjoy his BIG comfy c...,"04,June,2004",female,17,Sagittarius,indUnk
1,"Growing up, I never had the sense that I ever ...","21,June,2004",female,33,Cancer,Science
2,"In this paragraph, Hayek makes a terse and, it...","06,July,2004",male,23,Sagittarius,Student
3,urlLink fat man with lobster hands (billboa...,"12,July,2004",female,27,Leo,indUnk
4,Aww dude c'mon. CS is the greatest game ever c...,"24,July,2003",male,17,Gemini,Non-Profit


In [21]:
df_train["gender"].value_counts()/len(df_train)

gender
male      0.511
female    0.489
Name: count, dtype: float64

In [22]:
df_test["gender"].value_counts()/len(df_test)

gender
male      0.56
female    0.44
Name: count, dtype: float64

In [23]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.655161,0.570000
2,0.634400,0.656568,0.620000


TrainOutput(global_step=500, training_loss=0.6344190673828125, metrics={'train_runtime': 41.1893, 'train_samples_per_second': 48.556, 'train_steps_per_second': 12.139, 'total_flos': 526222110720000.0, 'train_loss': 0.6344190673828125, 'epoch': 2.0})

In [24]:
trainer.save_model("models/"+model_name + "_blog_authorship_gender")

In [26]:
classification_pipeline_gender = pipeline("text-classification", model="models/bert-base-uncased_blog_authorship_gender/checkpoint-500")

Device set to use cuda:0


In [27]:
classification_pipeline_gender("It is a sunny day outside. I want to go for a swim.")

[{'label': 'male', 'score': 0.562745213508606}]

In [29]:
classification_pipeline_gender("It is a sunny day outside. I want to make lemonade") # Model Bias?

[{'label': 'female', 'score': 0.5974498987197876}]

## Task1.2 - Gender Prediction (Binary Classification )
Choose a different encoder model and compare results.
Estimated time (20 mins)


# ToDo
1. Define model and training parameters
2. Perform training
3. Peer Learning: Which model did you choose and how did it perform?

## Task2 - Multi Class Classification:
Can you predict the age group of a person from a piece of written text?

In this dataset, the age column has a numeric value. We try to convert the age into a set of age groups say "<20","20-30" and "30+".

For ease of processing, we convert the datasets first to pandas datasets and then apply the processing needed to create the groups.

In [20]:
raw_df_train = raw_dataset["train"].to_pandas()
raw_df_test = raw_dataset["test"].to_pandas()

In [21]:
raw_df_train.head()

,text,date,gender,age,horoscope,job
0,We scared a seagull &amp; convinced a random d...,"29,August,2003",male,24,Gemini,Student
1,And on the really good news front: we visited ...,"30,May,2004",female,38,Virgo,indUnk
2,damnnnnn..chelsea snatched arjen robben from m...,"02,March,2004",male,16,Libra,Student
3,Author(s): Peter Mayle Genre: Fiction Revi...,"03,June,2004",female,38,Taurus,indUnk
4,You may be one of the few who actually comment...,"15,August,2004",male,27,Libra,indUnk


In [22]:
raw_df_train.age.value_counts().sort_index()

age
13     16
14     46
15     66
16    125
17    117
23     96
24    115
25    109
26     71
27     65
33     22
34     37
35     20
36     23
37     13
38     16
39      7
40      3
41      4
42      3
43      9
44      4
45      1
46      5
47      3
48      4
Name: count, dtype: int64

In [23]:
bins = [0, 20, 30, np.inf]
age_labels = ['<20', '20-30', '30+']
raw_df_train['AgeRange'] = pd.cut(raw_df_train['age'], bins, labels=age_labels)
raw_df_test['AgeRange'] = pd.cut(raw_df_test['age'], bins, labels=age_labels)
raw_df_train.head()

,text,date,gender,age,horoscope,job,AgeRange
0,We scared a seagull &amp; convinced a random d...,"29,August,2003",male,24,Gemini,Student,20-30
1,And on the really good news front: we visited ...,"30,May,2004",female,38,Virgo,indUnk,30+
2,damnnnnn..chelsea snatched arjen robben from m...,"02,March,2004",male,16,Libra,Student,<20
3,Author(s): Peter Mayle Genre: Fiction Revi...,"03,June,2004",female,38,Taurus,indUnk,30+
4,You may be one of the few who actually comment...,"15,August,2004",male,27,Libra,indUnk,20-30


In [24]:
raw_df_train.AgeRange.value_counts().sort_index()

AgeRange
<20      370
20-30    456
30+      174
Name: count, dtype: int64

We merge the datasets back into the DatasetDict object for huggingface and create the mappings id2label and label2id respectively

In [25]:
train_dataset = Dataset.from_dict(raw_df_train)
test_dataset = Dataset.from_dict(raw_df_test)
new_dataset = DatasetDict({"train":train_dataset,"test":test_dataset})

In [26]:
new_dataset["train"][0]

{'text': 'We scared a seagull &amp; convinced a random dude to take  urlLink a pic of us  with my digicam before going into what promised to be the actual sea port building.',
 'date': '29,August,2003',
 'gender': 'male',
 'age': 24,
 'horoscope': 'Gemini',
 'job': 'Student',
 'AgeRange': '20-30'}

In [27]:
id2label = {idx:label for idx, label in enumerate(age_labels)}
label2id = {label:idx for idx, label in enumerate(age_labels)}

The tokenize function is modified to account for the labels.

Note that the labels are in the form [<20, <20, 20-30.......30+].

However we need them to be in the format [[1,0,0],[1,0,0],[0,1,0],.....[0,0,1]] for training and evaluation.

Hence at first, we create a labels matrix of dimensions (batch_size, number_of_labels). Then we populate the matrix at row = batch_position and column = label_position with 0 or 1 depending on which label was present.

Finally we return the tokenize batch

In [28]:
def tokenize_function(batch, tokenizer, label2id, np ):#= tokenizer
  """ Function takes in a batch of data to tokenize and create the corresponding label matrix and returns the tokenized batch with labels
  """
  batch_size = len(batch["AgeRange"])
  tokenized_batch = tokenizer(batch["text"],padding=True, max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
  labels_matrix = np.zeros((batch_size, len(label2id))) #number of labels = len(label2id)
  for batch_pos,label in enumerate(batch["AgeRange"]):
      labels_matrix[batch_pos, label2id.get(label)] = 1
  tokenized_batch["labels"] = labels_matrix.tolist()
  return tokenized_batch

In [29]:
tokenized_dataset = new_dataset.map(tokenize_function, batched=True,
                                    fn_kwargs={"tokenizer": tokenizer, "label2id":label2id, "np":np},
                                    num_proc=4, remove_columns=raw_dataset['train'].column_names)
#use fn_kwargs to pass any arguments to the tokenizing function

Map (num_proc=4): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 529.03 examples/s]


In [30]:
tokenized_dataset.set_format(type="torch")

In [31]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding = True) #pads to the max sequence length in a batch

In [32]:
model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(age_labels), #number of classes
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    #f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    #roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {#'f1': f1_micro_average,
               #'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

In [50]:
training_args = TrainingArguments(
    output_dir="models/"+model_name+"_blog_authorship_age",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

In [51]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/scratch/local/33277935/ipykernel_2533073/552906714.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [52]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.465699,0.650000
2,0.402900,0.527951,0.620000
3,0.402900,0.543877,0.570000
4,0.194200,0.622845,0.570000
5,0.194200,0.617257,0.610000


TrainOutput(global_step=1250, training_loss=0.25565726928710936, metrics={'train_runtime': 78.4015, 'train_samples_per_second': 63.774, 'train_steps_per_second': 15.944, 'total_flos': 1315567088640000.0, 'train_loss': 0.25565726928710936, 'epoch': 5.0})

### Trying out the trained model

In [54]:
from transformers import pipeline

In [55]:
classification_pipeline = pipeline("text-classification", model="models/bert-base-uncased_blog_authorship_age/checkpoint-500")

Device set to use cuda:0


In [56]:
classification_pipeline("I am retired")

[{'label': '<20', 'score': 0.7296162247657776}]

In [57]:
classification_pipeline("I love my spouse and kids and love to take them outside after work.") # 

[{'label': '20-30', 'score': 0.7512363195419312}]

In [53]:
classification_pipeline("My friends and I want to go to Comic con") # 

[{'label': '20-30', 'score': 0.5268863439559937}]